Task 1: Regresión Lineal 

1.1 El coeficiente “3w2” representa cuanto cambia la variable dependiente “y” por cada unidad adicional de la característica "X2”. Siendo más específicos, por cada incremento de 1 en “X2”, “y "aumentara “3W2”
1.2 La multicolinealidad es cuando dos o más variables independientes están altamente relacionadas entre sí, causando que sea más dificultosa la tarea de saber cuál de ellos es realmente el responsable del cambio 


#TASK 2.0 Preparacion de dataset

Exploracion de datos inicial

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

filepath = "dataset_phishing.csv"
dataset = pd.read_csv(filepath)
#Exploracion de datos - Inciso 2.0
print("Distribución inicial de la columna 'status':")
print(dataset['status'].value_counts())
print("\nVerificacion de datos nulos")
print(dataset.isnull().sum())
print("\nResumen estadistico de los datos")
print(dataset.describe())




Distribución inicial de la columna 'status':
legitimate    5715
phishing      5715
Name: status, dtype: int64

Verificacion de datos nulos
url                0
length_url         0
length_hostname    0
ip                 0
nb_dots            0
                  ..
web_traffic        0
dns_record         0
google_index       0
page_rank          0
status             0
Length: 89, dtype: int64

Resumen estadistico de los datos
         length_url  length_hostname            ip       nb_dots  \
count  11430.000000     11430.000000  11430.000000  11430.000000   
mean      61.126684        21.090289      0.150569      2.480752   
std       55.297318        10.777171      0.357644      1.369686   
min       12.000000         4.000000      0.000000      1.000000   
25%       33.000000        15.000000      0.000000      2.000000   
50%       47.000000        19.000000      0.000000      2.000000   
75%       71.000000        24.000000      0.000000      3.000000   
max     1641.000000       2

Encoding de las columnas (Unicamente de la columna status)

In [ ]:
#Encoding de las columnas
# Ignorar la columna URL
if 'url' in dataset.columns:
    dataset = dataset.drop(columns=['url'])
# Codificar la columna status
if 'status' in dataset.columns:
    label_encoder = LabelEncoder()
    dataset['status'] = label_encoder.fit_transform(dataset['status'])

# Mostrar las primeras filas del dataset modificado
dataset.head(), dataset.info()

Revision de balanceo de dataset

In [ ]:
# Verificar la distribución de las clases en la columna 'status'
class_distribution = dataset['status'].value_counts()
class_distribution_percentage = dataset['status'].value_counts(normalize=True) * 100

class_distribution, class_distribution_percentage


Verificacion si es necesario el escalamiento de variables

In [ ]:
# Calcular el rango (mínimo y máximo) de las características numéricas
feature_ranges = dataset.drop(columns=['status']).agg(['min', 'max']).T
feature_ranges['range'] = feature_ranges['max'] - feature_ranges['min']
feature_ranges_sorted = feature_ranges.sort_values('range', ascending=False)
feature_ranges_sorted.head()




Escalamiento de variables

In [ ]:
from sklearn.preprocessing import StandardScaler

# Separar las características y la etiqueta
X = dataset.drop(columns=['status'])
y = dataset['status']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir X_scaled de nuevo a un DataFrame para mantener las columnas originales
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Mostrar las primeras filas del dataset escalado
X_scaled_df.head()
